# Задача 4: Минимальное число переводов для выравнивания расходов

Нужно по списку покупок определить переводы между участниками так, чтобы итоговые
затраты у всех стали равными (с точностью до копейки), а число переводов было минимальным.

Идея:
- считаем, сколько каждый потратил;
- вычисляем среднюю долю расходов;
- переводим избыточные суммы от тех, кто переплатил, к тем, кто недоплатил;
- минимизируем число переводов, соединяя «должников» и «кредиторов» жадно.


**Шаг 1. Считывание входных данных**

Формат:
- строка с именами участников;
- число N — количество покупок;
- далее N строк «имя сумма».

Для демонстрации используем пример из условия.


In [2]:
# Пример входных данных
input_text = "Ivan Aleksej Igor\n3\nIvan 500\nAleksej 100\nIvan 200\n"

lines = [line.strip() for line in input_text.strip().splitlines() if line.strip() != ""]
name_line = lines[0]
participants = name_line.split()
N = int(lines[1])
records = lines[2:2 + N]

print("participants =", participants)
print("N =", N)
print("records =", records)


participants = ['Ivan', 'Aleksej', 'Igor']
N = 3
records = ['Ivan 500', 'Aleksej 100', 'Ivan 200']


**Шаг 2. Суммируем расходы каждого участника**

Создадим словарь расходов и добавим суммы из записей покупок.


In [3]:
spent = {name: 0 for name in participants}
for rec in records:
    name, amount = rec.split()
    spent[name] += int(amount)

print("spent =", spent)


spent = {'Ivan': 700, 'Aleksej': 100, 'Igor': 0}


**Шаг 3. Находим среднюю долю расходов**

Каждый в итоге должен потратить одну и ту же сумму —
суммарные расходы делим поровну.


In [4]:
total = sum(spent.values())
count = len(participants)
average = total / count

print("total =", total)
print("average =", average)


total = 800
average = 266.6666666666667


**Шаг 4. Рассчитываем баланс каждого участника**

Баланс = потратил − средняя доля.
Если баланс > 0 — участник должен получить деньги,
если баланс < 0 — должен отдать.


In [5]:
balance = {name: spent[name] - average for name in participants}
print("balance =", balance)


balance = {'Ivan': 433.3333333333333, 'Aleksej': -166.66666666666669, 'Igor': -266.6666666666667}


**Шаг 5. Формируем списки кредиторов и должников**

Разделим участников на две группы:
- кредиторы (balance > 0)
- должники (balance < 0)


In [6]:
creditors = []  # (name, amount)
debtors = []    # (name, amount)

for name, b in balance.items():
    if b > 0:
        creditors.append([name, b])
    elif b < 0:
        debtors.append([name, -b])

print("creditors =", creditors)
print("debtors =", debtors)


creditors = [['Ivan', 433.3333333333333]]
debtors = [['Aleksej', 166.66666666666669], ['Igor', 266.6666666666667]]


**Шаг 6. Жадно строим минимальный набор переводов**

Идём по спискам должников и кредиторов и закрываем долги,
формируя переводы. Каждый перевод полностью закрывает
либо долг, либо кредит — это даёт минимальное число переводов.


In [7]:
transfers = []  # (from, to, amount)

i = 0
j = 0
while i < len(debtors) and j < len(creditors):
    d_name, d_amount = debtors[i]
    c_name, c_amount = creditors[j]

    pay = min(d_amount, c_amount)
    transfers.append((d_name, c_name, pay))

    d_amount -= pay
    c_amount -= pay

    debtors[i][1] = d_amount
    creditors[j][1] = c_amount

    if d_amount == 0:
        i += 1
    if c_amount == 0:
        j += 1

print("transfers =", transfers)


transfers = [('Aleksej', 'Ivan', 166.66666666666669), ('Igor', 'Ivan', 266.66666666666663)]


**Шаг 7. Вывод результата**

Печатаем количество переводов и сами переводы с округлением до 2 знаков.


In [8]:
print(len(transfers))
for frm, to, amount in transfers:
    print(frm, to, f"{amount:.2f}")


2
Aleksej Ivan 166.67
Igor Ivan 266.67
